In [1]:
from sympy import symbols
from IPython.display import display, Math
from kin_model import PhotoKineticSymbolicModel

In [2]:
str_model = """
BR -hv-> ^1BR --> BR          // k_S  # population of singlet state and decay to GS with rate k_S
^1BR --> ^3BR --> BR          // k_{isc} ; k_T
^3BR + ^3O_2 --> ^1O_2 + BR   // k_{TT}
^1O_2 --> ^3O_2               // k_d
^1O_2 + BR --> BR + ^3O_2     // k_q
BR + ^1O_2 -->                // k_r
"""

model = PhotoKineticSymbolicModel.from_text(str_model)
model.pprint_model()
print('\n')
model.pprint_equations()
print('\n')

# substitute for isc quantum yield
k_S, k_isc, *_ = model.symbols['rate_constants']
phi_isc = symbols('\phi_{isc}')
subs = [(k_isc / (k_isc + k_S), phi_isc)]

model.steady_state_approx(['^1BR', '^3BR', '^1O_2'], subs)  # set the species for SS approximation

<IPython.core.display.Math object>

Eq(Derivative(c_{BR}(t), t), -J + k_S*c_{^1BR}(t) + k_T*c_{^3BR}(t) - k_r*c_{BR}(t)*c_{^1O_2}(t) + k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t))

Eq(Derivative(c_{^1BR}(t), t), J - k_S*c_{^1BR}(t) - k_{isc}*c_{^1BR}(t))

Eq(Derivative(c_{^3BR}(t), t), -k_T*c_{^3BR}(t) - k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t) + k_{isc}*c_{^1BR}(t))

Eq(Derivative(c_{^3O_2}(t), t), k_d*c_{^1O_2}(t) + k_q*c_{BR}(t)*c_{^1O_2}(t) - k_{TT}*c_{^3BR}(t)*c_{^3O_2}(t))

...

Eq(Derivative(c_{BR}(t), t), -J*\phi_{isc}*k_r*k_{TT}*c_{BR}(t)*c_{^3O_2}(t)/((k_T + k_{TT}*c_{^3O_2}(t))*(k_d + k_q*c_{BR}(t) + k_r*c_{BR}(t))))

Eq(c_{^1BR}(t), J/(k_S + k_{isc}))

Eq(c_{^3BR}(t), J*\phi_{isc}/(k_T + k_{TT}*c_{^3O_2}(t)))

Eq(Derivative(c_{^3O_2}(t), t), -J*\phi_{isc}*k_r*k_{TT}*c_{BR}(t)*c_{^3O_2}(t)/((k_T + k_{TT}*c_{^3O_2}(t))*(k_d + k_q*c_{BR}(t) + k_r*c_{BR}(t))))

Eq(c_{^1O_2}(t), J*\phi_{isc}*k_{TT}*c_{^3O_2}(t)/((k_T + k_{TT}*c_{^3O_2}(t))*(k_d + k_q*c_{BR}(t) + k_r*c_{BR}(t))))

In [21]:
t, J = symbols('t J')   # J = 1/V * q0 * (1 - 10 ** (-l*eps*c_GS))

c_GS = Function('c_{BR}')(t)  # ground state
c_S = Function('c_{^1BR}')(t)  # singlet
c_T = Function('c_{^3BR}')(t)  # triplet
c_1O = Function('c_{^1O_2}')(t)  # singlet oxygen
c_3O = Function('c_{^3O_2}')(t)  # triplet oxygen

k_ST, k_SGS, k_TGS, k_TT = symbols('k_isc k_SGS k_TGS k_TT')
k_1O3O, k_r, k_q = symbols('k_1O3O k_r k_q')

q0, V, l, eps = symbols('q0 V l epsilon')


In [22]:
# differential equations
eq0 = Eq(c_GS.diff(t), -J + k_SGS*c_S + k_TGS*c_T - k_r * c_GS*c_1O + k_TT * c_3O*c_T)
eq1 = Eq(c_S.diff(t), +J -k_ST*c_S - k_SGS*c_S )
eq2 = Eq(c_T.diff(t), +k_ST*c_S -k_TGS*c_T - k_TT * c_3O*c_T)
eq3 = Eq(c_1O.diff(t), +k_TT * c_3O*c_T -k_1O3O*c_1O - k_r * c_GS*c_1O)

display(eq0)
display(eq1)
display(eq2)
display(eq3)

Eq(Derivative(c_{BR}(t), t), -J + k_SGS*c_{^1BR}(t) + k_TGS*c_{^3BR}(t) + k_TT*c_{^3BR}(t)*c_{^3O_2}(t) - k_r*c_{BR}(t)*c_{^1O_2}(t))

Eq(Derivative(c_{^1BR}(t), t), J - k_SGS*c_{^1BR}(t) - k_isc*c_{^1BR}(t))

Eq(Derivative(c_{^3BR}(t), t), -k_TGS*c_{^3BR}(t) - k_TT*c_{^3BR}(t)*c_{^3O_2}(t) + k_isc*c_{^1BR}(t))

Eq(Derivative(c_{^1O_2}(t), t), -k_1O3O*c_{^1O_2}(t) + k_TT*c_{^3BR}(t)*c_{^3O_2}(t) - k_r*c_{BR}(t)*c_{^1O_2}(t))

In [23]:
# solve the system of equations, make steady state approximation for singlet, triplet and singlet oxygen concentrations
solution = solve([eq0, Eq(eq1.rhs, 0), Eq(eq2.rhs, 0), Eq(eq3.rhs, 0)], [c_GS.diff(t), c_S, c_T, c_1O])

for key, value in solution.items():
    eq = Eq(key, value)
    display(factor(simplify(eq)).subs(k_ST / (k_ST + k_SGS), phi_isc))

Eq(Derivative(c_{BR}(t), t), -J*\phi_{isc}*k_TT*k_r*c_{BR}(t)*c_{^3O_2}(t)/((k_1O3O + k_r*c_{BR}(t))*(k_TGS + k_TT*c_{^3O_2}(t))))

Eq(c_{^1BR}(t), J/(k_SGS + k_isc))

Eq(c_{^3BR}(t), J*\phi_{isc}/(k_TGS + k_TT*c_{^3O_2}(t)))

Eq(c_{^1O_2}(t), J*\phi_{isc}*k_TT*c_{^3O_2}(t)/((k_1O3O + k_r*c_{BR}(t))*(k_TGS + k_TT*c_{^3O_2}(t))))